In [34]:
import numpy as np
import pandas as pd

In [35]:
df = pd.read_excel('FilteredViewTransactionsExpensesAll_CSILakiMIesNet_Eng.xlsx')
df.head(2)

,Parent Company / Emoyhtiö,Parent Company Type / Emoyhtiön tyyppi,Parent Company Home Country / Emoyhtiön kotimaa,Parent Company Industry / Emoyhtiön toimiala,Parent Company Responsible / Emoyhtiön asiakasvastaava,Principal / Toimeksiantaja,"Principal Type / Toimeksiantajan tyyppi (e.g. key customer, customer, partner, prospect, other)",Principal Home Country / Toimeksiantajan kotimaa,Payer / Maksaja,Payer Home Country / Maksajan kotimaa,...,Type (Fee/Expense) / Tyyppi (Palkkio/Kulu),User / Käyttäjä,Invoice Number / Laskunumero,Non-Billable / Ei laskuteta (False/True),Assignment.agency_guid,Assignment.Owner_Guid,Assignment.Responsible_Person_Guid,Assignment.Record_Guid,User.Record_Guid,Users.Manager_Guid
0,NaN,NaN,NaN,NaN,NaN,Ankallispankki Oy,Prospect,Suomi,NaN,NaN,...,Palkkio,deb049ee-93d0-49df-8aea-1045c63a90f2,NaN,False,faa6df72-06be-4eaf-999e-022ef49754eb,deb049ee-93d0-49df-8aea-1045c63a90f2,deb049ee-93d0-49df-8aea-1045c63a90f2,11ad526c-0d72-445c-abd7-09ac5590c45f,deb049ee-93d0-49df-8aea-1045c63a90f2,NaN
1,NaN,NaN,NaN,NaN,NaN,Ankallispankki Oy,Prospect,Suomi,NaN,NaN,...,Palkkio,deb049ee-93d0-49df-8aea-1045c63a90f2,NaN,False,faa6df72-06be-4eaf-999e-022ef49754eb,deb049ee-93d0-49df-8aea-1045c63a90f2,deb049ee-93d0-49df-8aea-1045c63a90f2,11ad526c-0d72-445c-abd7-09ac5590c45f,deb049ee-93d0-49df-8aea-1045c63a90f2,NaN


In [36]:
date_columns = ['Date / Päiväys', 'Entry Date / Kirjauspäivä', 'Invoicing Date / Laskutuspäivä', 'Payment Date / Maksupäivä']
# for colname in df.columns:
#     if df[colname].dtype == object:
#         df[colname] = df[colname].fillna('nan')
#     elif df[colname].dtype in [int, float]:
#         df[colname] = df[colname].fillna(-100)
#     elif colname in date_columns:
#         df[colname] = df[colname].fillna('nan')

In [37]:
df[date_columns] = df[date_columns].astype(str)
for date_column in date_columns:
    df[date_column] = df[date_column].apply(lambda x: str(x)[:10])

In [38]:
# import seaborn as sns
# %matplotlib inline
# from datetime import datetime
# import time

# DATE_FORMAT = '%Y-%m-%d'
# df['date'] = df['Date / Päiväys'].apply(lambda x: datetime.strptime(str(x)[:10], DATE_FORMAT))
# df['timestamp'] = df['date'].apply(lambda x: time.mktime(x.timetuple()))
# sns.distplot(df['timestamp'], bins = 15)

In [39]:
new_columns = [column.split('/')[0][:-1] for column in df.columns]

In [40]:
df.columns = new_columns

In [41]:
bool_columns = ['Internal', 'Non-Billable']

for colname in bool_columns:
    df[colname] = df[colname].apply(lambda x: 'True' if x else 'False')

In [42]:
df['Assignment'] = df['Assignment'].apply(lambda x: x.split()[0])

In [43]:
non_use = ['Business Unit', 'Business Area', 'Job Title', 'Manager', 'Account', 'Assignment.agency_gui',
           'Assignment.Owner_Gui', 'Assignment.Responsible_Person_Gui','Assignment.Record_Gui', 'User',
           'User.Record_Gui', 'Users.Manager_Gui', 'Registered']
df = df[[x for x in df.columns if x not in non_use]]

In [44]:
common_cols =  ['Parent Company', 'Parent Company Type', 'Parent Company Home Country',
       'Parent Company Industry', 'Parent Company Responsible', 'Principal',
       'Principal Type', 'Principal Home Country', 'Payer',
       'Payer Home Country', 'Industry', 'Customer Responsible',
       'Owner', 'Responsible', 'Seller', 'Cost Center'] + ['Assignment Type', 'Author', 'Category', 'Employee Role'] +\
['Internal']

In [45]:
uncommon_cols = [col for col in df.columns if col not in common_cols + ['Assignment', 'Transaction Type']]

In [46]:
def common_agg(rec):
    key = rec['Assignment']
    

In [47]:
df_common = df[['Assignment'] + common_cols].drop_duplicates(['Assignment']).set_index('Assignment')

In [48]:
def aggregator(rec):
#     print(rec[['Billable Hours', 'Status (Registere', 'Paid', 'Invoiced', 'Margin', 'Invoicing Date', 'Total']])
    max_index = np.max(list(rec.index))
    result = pd.Series(rec.loc[max_index, :])
    result['Billable Hours'] = np.sum(rec['Billable Hours'])
    result['Hours Worked'] = np.sum(rec['Hours Worked'])
    if 'Maksettu' in rec['Status (Registere']:
        paid = np.sum(rec['Paid'])
        result['Paid'] = paid
    
    if 'Laskutettu' in rec['Status (Registere']:
        invoiced = np.sum(rec['Invoiced'])
        result['Invoiced'] = invoiced
        result['Margin'] = np.sum(rec['Margin'])
    return result[uncommon_cols].to_dict()

In [49]:
result = df.groupby(['Assignment', 'Transaction'], as_index=True).apply(lambda x: aggregator(x))

In [50]:
result = result.reset_index()

In [51]:
result_json = {}
def anton_agg(rec):
#     print(rec)
    key = rec['Assignment'].iloc[0]
#     rec = rec.drop(['Assignment'], axis=1)
    result = {rec.loc[i,'Transaction']: rec.loc[i,0] for i in rec.index}
#     print(rec.loc[rec.index[0],0])
#     output = pd.Series(result, name = key)
    output = {
        key: result,
    }
#     print(output)
    result_json.update(output)
#     return output

In [52]:
# result.rename{}

In [53]:
after_group_by = result.groupby(['Assignment'], as_index=False).agg(anton_agg)

In [54]:
# result_json['1']['Esiintyminen oikeudessa / Toijalan käräjäoikeus']

In [55]:
# df_uncommon = pd.DataFrame.from_dict(result_json)

In [56]:
series_uncommon = pd.Series(result_json)

In [57]:
df_prefinal = pd.concat([df_common, series_uncommon], axis = 1)
df_prefinal = df_prefinal.rename(columns = {0: 'Transactions'})

In [58]:
json_0 = df_prefinal.T.to_dict()

In [59]:
import json

In [60]:
json_0

{'1': {'Assignment Type': 'Kilpailuoikeus',
  'Author': 'Helen Hakaniemi',
  'Category': nan,
  'Cost Center': nan,
  'Customer Responsible': 'Helen Hakaniemi',
  'Employee Role': nan,
  'Industry': 'Muu',
  'Internal': 'False',
  'Owner': 'Helen Hakaniemi',
  'Parent Company': nan,
  'Parent Company Home Country': nan,
  'Parent Company Industry': nan,
  'Parent Company Responsible': nan,
  'Parent Company Type': nan,
  'Payer': nan,
  'Payer Home Country': nan,
  'Principal': 'AsuntoOy Ankka Linna',
  'Principal Home Country': 'Suomi',
  'Principal Type': 'Prospect',
  'Responsible': 'Boris Keilaniemi',
  'Seller': nan,
  'Transactions': {'Esiintyminen oikeudessa / Toijalan käräjäoikeus': {'Billable Hours': 7.5,
    'Cost Center (Business Unit)': nan,
    'Date': '2008-09-03',
    'Entry Date': '2008-09-01',
    'Hour Price': 170.0,
    'Hours Worked': 7.5,
    'Invoice Number': '1',
    'Invoiced': nan,
    'Invoicing Date': '2008-09-03',
    'Margin': 8.2699999999999996,
    'Non-B

In [61]:
json.dump(json_0, open('json_0', 'w'))

In [63]:
result[0][0]

{'Billable Hours': 7.5,
 'Cost Center (Business Unit)': nan,
 'Date': '2008-09-03',
 'Entry Date': '2008-09-01',
 'Hour Price': 170.0,
 'Hours Worked': 7.5,
 'Invoice Number': '1',
 'Invoiced': nan,
 'Invoicing Date': '2008-09-03',
 'Margin': 8.2699999999999996,
 'Non-Billable': 'False',
 'Paid': 10.039999999999999,
 'Payment Date': '2008-09-03',
 'Skill': nan,
 'Status (Registere': 'Maksettu',
 'Tax Rate': 'Alv 22 %',
 'Total': 10.039999999999999,
 'Transaction': 'Esiintyminen oikeudessa / Toijalan käräjäoikeus',
 'Transaction Type Englis': 'Court appearance',
 'Transaction status (Ope': 'Maksettu',
 'Type (Fe': 'Palkkio'}

In [33]:
# df['Status (Registere'].value_counts()

In [34]:
# status_map_dict = {
#     'Laskutettu': 1,
#     'Kirjattu': 0,
    
# }


# def get_maximus_status():
#     return

In [17]:
df_agg = result.agg(
    {
        'Status (Registere': np.max
    }
)

In [18]:
df_last_statuses = pd.merge(df, df_agg, how = 'inner', on = list(df_agg.columns))

In [19]:
df_last_statuses.shape

(2836, 44)

In [47]:
df_agg['Status (Registere'].value_counts()

Maksettu      560
Laskutettu    333
Kirjattu      267
Name: Status (Registere, dtype: int64

In [48]:
df_agg['Status (Registere'].isnull().sum()

0

In [55]:
df_agg.iloc[0,:].values

array(['1 - Osakekauppariita',
       'Esiintyminen oikeudessa / Toijalan käräjäoikeus', 'Maksettu'], dtype=object)

In [ ]:
df_last_statuses.loc[]

In [60]:
tmp = df_last_statuses[['Assignment', 'Transaction', 'Status (Registere']]

In [120]:
import pprint

In [123]:
pprint.pprint(result[0])

{'1': {'Esiintyminen oikeudessa': {'Assignment Type': 'Kilpailuoikeus',
                                   'Author': 'Helen Hakaniemi',
                                   'Billable Hours': 0.06,
                                   'Category': nan,
                                   'Cost Center (Business Unit)': nan,
                                   'Date': Timestamp('2008-09-03 00:00:00'),
                                   'Employee Role': nan,
                                   'Entry Date': Timestamp('2008-09-01 00:00:00'),
                                   'Hour Price': 170.0,
                                   'Hours Worked': 0.06,
                                   'Internal': False,
                                   'Invoice Number': '1',
                                   'Invoiced': nan,
                                   'Invoicing Date': Timestamp('2008-09-03 00:00:00'),
                                   'Margin': 8.27,
                                   'Non-Billable':

In [119]:
import json

In [ ]:
json.dum

In [7]:
for colname in df.columns:
    # print(df['Principal / Toimeksiantaja'].value_counts()()
    value_counts = df[colname].value_counts()
    print(colname, '\n', 'n different values is', len(value_counts), '\nmost popular is\n', value_counts.head(10), '\n number of nans is', df[colname].isnull().sum())
    print()

Parent Company 
 n different values is 6 
most popular is
 Kolmio Oy                494
Ankallispankki Oy         58
Tiinan Muonitus Oy        53
Leivolan Leivokset Oy      9
Hannu Hanhi Oy             4
A yhtiö Oy                 2
Name: Parent Company, dtype: int64 
 number of nans is 4617

Parent Company Type 
 n different values is 2 
most popular is
 Asiakas       547
Osakeyhtiö     62
Name: Parent Company Type, dtype: int64 
 number of nans is 4628

Parent Company Home Country 
 n different values is 1 
most popular is
 Suomi    620
Name: Parent Company Home Country, dtype: int64 
 number of nans is 4617

Parent Company Industry 
 n different values is 1 
most popular is
 Pankki    60
Name: Parent Company Industry, dtype: int64 
 number of nans is 5177

Parent Company Responsible 
 n different values is 5 
most popular is
 Nelli Niittykumpu     494
Helen Hakaniemi        58
Viivi Vuosaari         53
Lenita Lauttasaari      9
Kustaa Kulosaari        6
Name: Parent Company Responsi